 Set debug dot directory env variable.

In [ ]:
nb = "nb4"
dotdir = "/home/root/gst-dot/" + nb

# create directory for dot files
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

This step imports required python packages for this demo and initializes the GStreamer library.

In [ ]:
from IPython.display import Image, display, clear_output
import sys
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

Gst.init(None);

Enable debug and set debug level.

In [ ]:
Gst.debug_set_active(False)
Gst.debug_set_default_threshold(3)

First pipeline

In [ ]:
# v4l2src element
io_mode = "mmap"
device = "/dev/video0"
src_1 = "v4l2src device=" + device + " io-mode=" + io_mode

# caps filter element
width = 1280
height = 720
fmt = "YUY2"
caps = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt 

# kmssink element (first plane)
driver_name = "xlnx"
plane_id_1 = 34
xoff_1 = 0
yoff_1 = 0
render_rectangle_1 = "<" + str(xoff_1) + "," + str(yoff_1) + "," + str(width) + "," + str(height) + ">"
sink_1 = "kmssink" + " driver-name=" + driver_name + " plane-id=" + str(plane_id_1) + " render-rectangle=" + render_rectangle_1

# first pipeline
pipe_1 = src_1 + " ! " + caps + " ! " + sink_1

Second pipeline.

In [ ]:
# mulitfilesrc element
file_name = "/usr/share/movies/Big_Buck_Bunny_4K.webm.360p.vp9.webm"
loop = True
src_2 = "multifilesrc location=" + file_name + " loop=" + str(loop)

# decode bin element
dec = "decodebin"

# videoconvert element
cvt = "videoconvert"

# kmssink element (second plane)
driver_name = "xlnx"
plane_id_2 = 35
xoff_2 = 0
yoff_2 = 720
width_2 = 640
height_2 = 360
render_rectangle_2 = "<" + str(xoff_2) + "," + str(yoff_2) + "," + str(width_2) + "," + str(height_2) + ">"
sink_2 = "kmssink" + " driver-name=" + driver_name + " plane-id=" + str(plane_id_2) + " render-rectangle=" + render_rectangle_2

# second pipeline
pipe_2 = src_2 + " ! " + dec + " ! " + cvt + " ! "+ sink_2

Parse first and second pipeline as single graph.

In [ ]:
pipeline = Gst.parse_launch(pipe_1 + " " + pipe_2)

This bus_call function listens on the bus for EOS and Error events.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline. Create main loop and listen to bus messages. Display the video frames in the notebook.

To stop the pipeline, click the square shaped 'interrupt kerenl' icon in the top menu bar. Creates a dot graph before stopping the pipeline.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

Show pipeline graph. This step may take a few seconds. Double click on the generate image file that shows the graph to zoom in.

In [ ]:
import pydot

# register dot plugins for png export to work
!dot -c

# convert dot file to png and display
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

The end.